In [ ]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import keras
import numpy as np
import tensorflow as tf
from keras.applications.resnet50 import ResNet50, preprocess_input as resnet_preprocess, decode_predictions as resnet_decode
from tensorflow.keras.applications.inception_v3 import InceptionV3, preprocess_input as inception_preprocess, decode_predictions as inception_decode
from tensorflow.keras.preprocessing import image
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

In [ ]:
# Constants
IMG_PATH = '/content/drive/MyDrive/FB_IMG_1651602337687.jpg'
IMG_SIZE_RESNET = (224, 224)
IMG_SIZE_INCEPTION = (299, 299)
NUM_CLASSES = 10
BATCH_SIZE = 32
LEARNING_RATE = 0.0001
FINE_TUNE_LR = 0.00001
EPOCHS = 2

In [ ]:
# ResNet50 Prediction Function
def predict_resnet(img_path):
    model = ResNet50(weights='imagenet')

    img = keras.utils.load_img(img_path, target_size=IMG_SIZE_RESNET)
    x = keras.utils.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = resnet_preprocess(x)

    preds = model.predict(x)
    print('ResNet50 Predicted:', resnet_decode(preds, top=3)[0])

In [ ]:
# InceptionV3 Transfer Learning Function
def apply_transfer_learning(epochs=EPOCHS, batch_size=BATCH_SIZE):
    # Load base InceptionV3 model
    base_model = InceptionV3(weights='imagenet', include_top=False)

    # Add custom layers for CIFAR-10 classification
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(1024, activation='relu')(x)
    predictions = Dense(NUM_CLASSES, activation='softmax')(x)

    model = Model(inputs=base_model.input, outputs=predictions)

    # Load CIFAR-10 dataset
    (x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()

    # Resize CIFAR-10 images to 299x299 (InceptionV3 input size)
    def resize_images(images): # This line was incorrectly indented
        return tf.image.resize(images, IMG_SIZE_INCEPTION)

    x_train = resize_images(x_train)
    x_test = resize_images(x_test)

    # Normalize data between -1 and 1
    x_train = inception_preprocess(x_train)
    x_test = inception_preprocess(x_test)

    # Convert labels to one-hot encoding
    y_train = tf.keras.utils.to_categorical(y_train, NUM_CLASSES)
    y_test = tf.keras.utils.to_categorical(y_test, NUM_CLASSES)

    # Convert arrays to TensorFlow datasets
    train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train)).batch(batch_size).shuffle(10000)
    val_dataset = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(batch_size)

    # Freeze base model layers and compile
    for layer in base_model.layers:
        layer.trainable = False

    model.compile(optimizer=Adam(learning_rate=LEARNING_RATE), loss='categorical_crossentropy', metrics=['accuracy'])

    # Train the newly added layers
    model.fit(train_dataset, epochs=epochs, validation_data=val_dataset)

    # Unfreeze some layers for fine-tuning
    for layer in base_model.layers[-20:]:
        layer.trainable = True

    # Recompile with lower learning rate for fine-tuning
    model.compile(optimizer=Adam(learning_rate=FINE_TUNE_LR), loss='categorical_crossentropy', metrics=['accuracy'])

    # Fine-tune the model
    model.fit(train_dataset, epochs=epochs, validation_data=validation_dataset)


In [ ]:
# InceptionV3 Prediction Function
def predict_inception(img_path):
    model = InceptionV3(weights='imagenet')

    img = keras.utils.load_img(img_path, target_size=IMG_SIZE_INCEPTION)
    x = keras.utils.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = inception_preprocess(x)

    preds = model.predict(x)
    print('InceptionV3 Predicted:', inception_decode(preds, top=3)[0])

In [ ]:
# Main Execution
if __name__ == "__main__":
    # Predict using ResNet50
    predict_resnet(IMG_PATH)

    # Predict using InceptionV3 without transfer learning
    predict_inception(IMG_PATH)

    # Apply transfer learning on InceptionV3 for CIFAR-10
    apply_transfer_learning()

1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step
ResNet50 Predicted: [('n04355933', 'sunglass', 0.64635277), ('n04356056', 'sunglasses', 0.065014146), ('n03250847', 'drumstick', 0.055747874)]
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step
InceptionV3 Predicted: [('n04355933', 'sunglass', 0.5776342), ('n04356056', 'sunglasses', 0.41522706), ('n03763968', 'military_uniform', 0.0013891654)]
